In [3]:
import pandas as pd
import numpy as np

# Add rpy2 to run R code chunks
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [4]:
%%R 

# Load packages needed for visualization in R
suppressPackageStartupMessages({
    library(colorspace)
    library(cowplot)
    library(tidyverse)
    library(patchwork)
    library(viridis)
    library(fsbrain)
    library(glue)


    # Set cowplot theme
    theme_set(theme_cowplot())
})

In [5]:
%%R 

# Define directories
subjects_dir = "~/data/fs/"
github_dir <- "~/github/MEG_functional_connectivity/"

# fsaverage surface
subject_id = 'fsaverage'

############ SCHAEFER100 ##############
# 1. Category-selective: "lateraloccipital", "fusiform"
# 2. Visual: "pericalcarine", "lingual"
# 3. Prefrontal cortex: "7Networks_LH_Cont_PFCl_1", "7Networks_RH_Cont_PFCl_1"
# 4. Intraparietal sulcus: "7Networks_LH_Cont_PFCl_1", "7Networks_RH_Cont_PFCl_1"

# Schaefer100 left/right surface annot
schaef_atlas_dir = "~/data/neuroimaging_atlases/surfaces/Schaefer/fsaverage/"
lh_schaef_annot = freesurferformats::read.fs.annot(glue("{schaef_atlas_dir}/atl-Schaefer2018_space-fsaverage_hemi-L_desc-100Parcels7Networks_deterministic.annot"))
rh_schaef_annot = freesurferformats::read.fs.annot(glue("{schaef_atlas_dir}/atl-Schaefer2018_space-fsaverage_hemi-R_desc-100Parcels7Networks_deterministic.annot"))

lh_schaef_parcels_of_interest <- case_when(lh_schaef_annot$label_names %in% c("7Networks_LH_DorsAttn_Post_1") ~ 1,
                                           lh_schaef_annot$label_names %in% c("7Networks_LH_Vis_4", "7Networks_LH_Vis_5") ~ 2,
                                           lh_schaef_annot$label_names %in% c("7Networks_LH_SalVentAttn_PFCl_1") ~ 3,
                                           lh_schaef_annot$label_names %in% c("7Networks_LH_DorsAttn_Post_3") ~ 4,
                                           T ~ NA_real_)
rh_schaef_parcels_of_interest <- case_when(rh_schaef_annot$label_names %in% c("7Networks_RH_Vis_3") ~ 1,
                                           rh_schaef_annot$label_names %in% c("7Networks_RH_Vis_4", "7Networks_RH_Vis_5") ~ 2,
                                           rh_schaef_annot$label_names %in% c("7Networks_RH_Cont_PFCl_3") ~ 3,
                                           rh_schaef_annot$label_names %in% c("7Networks_RH_DorsAttn_Post") ~ 4,
                                           T ~ NA_real_)

schaefer_atlas_rois_on_surface <- vis.symmetric.data.on.subject(subjects_dir, vis_subject_id=subject_id, 
                              morph_data_lh=lh_schaef_parcels_of_interest, 
                              morph_data_rh=rh_schaef_parcels_of_interest, 
                              bg="sulc_light",
                              makecmap_options = list('colFn'=viridis),
                              surface="inflated", draw_colorbar = T,
                              rglactions = list('shift_hemis_apart'=TRUE))
# Export the vis
export(schaefer_atlas_rois_on_surface, img_only = TRUE, 
       output_img = glue("{github_dir}/plots/methods/Schaefer100_atlas_ROIs.png"),
       rglactions = list('shift_hemis_apart'=TRUE,'no_vis'=T), 
       view_angles=fsbrain::get.view.angle.names(angle_set = "t9"))


# A tibble: 1 × 7
  format width height colorspace matte filesize density
  <chr>  <int>  <int> <chr>      <lgl>    <int> <chr>  
1 PNG     2038   1986 sRGB       FALSE        0 72x72  


In [4]:
%%R 

cols <- c("#f9796f", "#efab5a", "#ff78ba", "#c77cff"
)
# Iterate from 1 to 12, and for each number, simulate 50 random values; plot as a one-row heatmap with the corresponding color
set.seed(1)

plot_list <- list()
for (i in 1:4) {
    y_data <- data.frame(val=rnorm(250), index=1:250)
    p <- ggplot(y_data, aes(x=index, y=0, fill=val)) + 
        geom_tile() + 
        scale_fill_gradient(low="white", high=cols[i]) +
        theme_void() +
        theme(legend.position="none")

    plot_list[[i]] <- p
}

wrap_plots(plot_list, ncol=1)
ggsave("../plots/methods/example_time_series.svg", width=3, height=2, units="in", dpi=300)